Chinmay Pimpalkhare (200100115)


# Text Generation using GPT-2


We shall be making use of GPT-2 which will be fine-tuned on the task of text generation. We shall try to see how it does on various forms of text.


In [1]:
!nvidia-smi #Checking the GPU status

Wed Aug 16 08:57:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Here we are installing all the necessary required libraries such as the transformers library from Hugging Face

In [2]:
! pip install -q transformers accelerate sentencepiece gradio #Installing the important libraries

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 13.4 MB/s eta 0:00:

We shall also be needing the datasets library from HuggingFace so we are also installing that separately.

In [3]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


We are using the "Yelp Reviews" dataset right now. We shall switch to some other task for more finetuning


In [5]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [9]:
dataset["test"][200] #Here we can check the descriptions of various contents of the dataset which we are using

{'label': 4,
 'text': "i have been getting my hair cut here for 15 years.  Marjorie is amazing and priceless.  I am as picky about my hairstylist as i am my dentist or doctor.  I travel extensively for work and get compliments no matter where i travel.  Wouldn't go anywhere else!"}

here we have our tokenization function. We are using the AutoTokenizer which is provided. We can experiment with different models as well. The padding and the truncation allow us to handle variable sequence lengths.

In [10]:
#Defining the tokenize function, which tokenizes the examples which we pass through it
def tokenize_function(examples):
  return tokenizer(examples["text"], padding = "max_length", truncation = True)

The map function allows us to preprocess the entire dataset in just one step. However, for the finetuning task, we can also instead create batches, and by random shuffling pass the data inside batches.

In [13]:
tokenized_datasets = dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Here we are making batches manually so that it is possible to train data on that more quickly.

In [14]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))

In [15]:
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Next, we need to train the hyperparameters for this model so that it can perform our task more efficiently. Hence, we shall do that next. For that we are creating a TrainingArguments class, we are also deciding where we want to store those outputs by specifying the output directory.

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

Next up we need to perform the evaluation task. For that we need to pass the trainer function first. We would be making use of the evaluate library task for that matter.

In [18]:
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.9 MB/s eta 0:00:00


In [19]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

Next, we write a function to convert the predictions to logits, because all Hugging Face transformers act with logits.

In [20]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

We also want to monitor all the evaluation metrics during the training process. We shall then specify the evaluation strategy for that.

In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

Finally we shall want to create a Trainer object which shall perform the training work for us. We pass all the specified quantities to this Trainer object

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Finally we shall finetune our model

In [23]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.147539,0.493000
2,No log,1.046712,0.570000
3,No log,1.074833,0.598000


TrainOutput(global_step=375, training_loss=0.90557666015625, metrics={'train_runtime': 369.6987, 'train_samples_per_second': 8.115, 'train_steps_per_second': 1.014, 'total_flos': 789354427392000.0, 'train_loss': 0.90557666015625, 'epoch': 3.0})

## **GUI with Gradio**
Here we shall build an interface for our model using Gradio.
We shall also publish it so that it can be used by anyone we wish to share it with

Here we are adding a GUI for the text generation model. We are using Gradio for the GUI.

In [ ]:
import gradio as gr

def generate_text(prompt, top_k, max_length, temperature):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        num_return_sequences=1
    )
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

inputs = [
    gr.inputs.Textbox(label="Prompt"),
    gr.inputs.Number(label="Top k", default=50),
    gr.inputs.Number(label="Max Length", default=200),
    gr.inputs.Slider(label="Temperature", default=0.7, minimum=0, maximum=1)
]
output = gr.outputs.Textbox(label="Generated Text")

interface = gr.Interface(
    fn=generate_text,
    inputs=inputs,
    outputs=output,
    title="Text Generation Interface",
    description="Generate text using a placeholder function",
    live=True
)

if __name__ == "__main__":
    interface.launch(debug = False
                    )


<ipython-input-30-ab92b4802c34>:16: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Textbox(label="Prompt"),
<ipython-input-30-ab92b4802c34>:16: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Prompt"),
<ipython-input-30-ab92b4802c34>:16: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  gr.inputs.Textbox(label="Prompt"),
<ipython-input-30-ab92b4802c34>:17: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Number(label="Top k", default=50),
<ipython-input-30-ab92b4802c34>:17: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Number(label="Top k", default=50),
<ipython-input-30-ab92b4802c34>:18: GradioDeprecation

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/routes.py", line 488, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1431, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1109, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    result = context.run(func, *args)
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 706, in wrapper
    response = f(*args, **kwargs)
  Fi